In [ ]:
# INITIAL DATA CLEANING AND ENLISTING
import pandas as pd
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
pd.set_option('display.max_columns', None)

# INCORPORATING DATASET FILE
file = open('./disease_components.csv', 'rb')
de = pd.read_csv(file, encoding='Windows-1252')
filex = open('./disease_cleaned.csv', 'w')

# adding new column and saving
de["info-symptoms"] = ""
de["info-causes"] = ""
de["info-medication"] = ""

In [ ]:
attributes = ["Name", "Overview", "Symptoms", "Causes", "Risk factors", "Diagnosis", "Treatment", "Remedies", "Medication"
]
bulletAttr = ["Symptoms", "Causes", "Medication"]
for row in range(0, 1183):
  for chosen in attributes:

    # SKIP IF N/A
    if pd.isna(de.loc[row, chosen]):
      continue

    # LOAD THE DATA
    txt = de.loc[row, chosen]

    # CHECK IF NAME
    if chosen=="Name":
      words = word_tokenize(txt)
      final = []
      for word in range(len(words)):
        if (words[word] =="(" and (words[word+1]=="See" or words[word+1]=="see")) or ("See" in words[word] or "see" in words[word]):
          break
        final.append(words[word])
      de.loc[row, chosen] = ' '.join(final)
      continue
    
    # CLEANING THE TEXT
    txt = list(txt)
    unwanted  = ["[", "]"]
    apos = ['s', 'm', 'r', 't']
    for char in range(len(txt)):
      if txt[char] == "$" and (txt[char+1] == "," or txt[char+1] == "\"" or txt[char+1] == "'"):
        txt[char+1] = "$"
      if txt[char] == "," and (txt[char-1] == "." or txt[char-1] == "\""):
        txt[char] = "$"
      if txt[char] in unwanted:
        txt[char] = "$"
      if txt[char] == "'" and char!=len(txt)-1:
        if (txt[char+1] in apos):
          g = 6
        else: 
          txt[char] = "$"
        if (txt[char+1]==","):
          txt[char+1] = "$"
      if txt[char] == "'" and char==len(txt)-1:
        txt[char] = "$"
    txt = "".join(txt)
    txt = txt.replace("\"", "")
    txt = txt.replace("''", "")
    txt = txt.replace("$", "")
    txt = txt.replace("\\n", "")
    txt = txt.replace("\\", "")

    # Stop-words
    stopWords = list(stopwords.words("english"))

    # Remove duplicates and unwanted sentences
    duplicates = []
    cleaned = []
    sentences = sent_tokenize(txt)
    for s in sentences:
      if chosen=="Symptoms" or chosen=="Causes": #or chosen=="Causes":
        if len(word_tokenize(s))>30:
          #print("Reomved: ", s)
          continue
      if s in cleaned:
        if s in duplicates:
            continue
        else:
            duplicates.append(s)
      else:
        if "doi" in s or "Accessed" in s or "Mayo Clinic" in s or "http" in s:
          continue
        else:
          cleaned.append(s)

    # New cleaned text
    tx = " ".join(cleaned)

    # Tokenize into sentences
    sent = sent_tokenize(tx)

    # Getting attributes for the info-box
    if chosen in bulletAttr:
      bullets = []
      for s in sent:
        flag = 1
        words = word_tokenize(s)
        for w in words:
          if w in stopWords:
            flag = 0
            break
        if flag and len(words)<=3:
          bullets.append(s)
      
      info = ""
      if bullets:
        for b in range(len(bullets)):
          bullets[b] = bullets[b].replace(".", "")
        info = ", ".join(bullets)
      
      if chosen=="Symptoms":
        de.loc[row, "info-symptoms"] = info
      elif chosen=="Causes":
        de.loc[row, "info-causes"] = info
      elif chosen=="Medication":
        de.loc[row, "info-medication"] = info
      print(chosen, " : ", info)
  
    # Summarizing the paragraphs
    words = word_tokenize(tx)
   
    # Creating a frequency table to keep the score of each word
      
    freqTable = dict()
    for word in words:
        word = word.lower()
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1
      
    # Creating a dictionary to keep the score of each sentence
    sentences = sent_tokenize(tx)
    sentenceValue = dict()

    if not sentences:
      de.loc[row, chosen] = ""
      continue
      
    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceValue:
                    sentenceValue[sentence] += freq
                else:
                    sentenceValue[sentence] = freq
      
      
      
    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue[sentence]
      
    # Average value of a sentence from the original text
    
    average = int(sumValues / len(sentenceValue))
      
    # Storing sentences into our summary.
    summary = ''
    if chosen=="Overview":
      summary += " " + sentences[0]
    
    factor = 1.3
    if chosen=="Symptoms" or chosen=="Causes":
      factor = 1.25

    for sentence in sentences:
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (factor * average)):
          if chosen=="Overview" and sentence == sentences[0]:
            continue
          else:
            summary += " " + sentence


    # Structure the language of the summary
    #summary = summary.lower()
    summary = summary.replace("your", "the")
    summary = summary.replace("you're", "one is")
    summary = summary.replace("you are", "one is")
    summary = summary.replace("you have", "one has")
    summary = summary.replace("you'll", "one'll")
    summary = summary.replace("you", "one")
    summary = summary.replace("Your", "The")
    summary = summary.replace("You're", "One is")
    summary = summary.replace("You are", "One is")
    summary = summary.replace("You have", "One has")
    summary = summary.replace("You'll", "One'll")
    summary = summary.replace("You", "One")
    summary = summary.replace("Onell", "One will")
    summary = summary.replace("oneve", "one has")
    summary = summary.replace("onere", "one is")
    summary = summary.replace("onell", "one will")
    summary = summary.replace("Oneve", "One has")
    summary = summary.replace("Onere", "One is")
    summary = summary.replace("Oneng", "Young")
    summary = summary.replace("oneng", "young")
    summary = summary.replace("include:", "include")
    summary = summary.replace("including:", "including")
    summary = summary.replace("ï¿½", "")
    summary = summary.replace("try these tips:", "")
    summary = summary.strip()
    summary = ' '.join(summary.split())
    if ". " not in summary:
      summary = ""
    # if (chosen=="Symptoms" or chosen=="Causes" or chosen=="Risk Factors") and summary!="":
    #   sts = sent_tokenize(summary)
    #   for senten in sts:
    #     words = word_tokenize(senten)
    #     # print(words)
    #     if len(words)>33:
    #       summary = ""
    #       break

    # if chosen=="Symptoms" or chosen=="Causes":
    print(row)
    print("\n")

    de.loc[row, chosen] = summary

In [ ]:
# Saving the cleaned data
de.to_csv(filex, index=False)

In [ ]:
print(de.loc[0, "Symptoms"])